In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

# Coattention encoder

We've constructed encoding matrices for the documents and questions, D and Q. Each document (e.g. paragraph in SQUAD) has its own D and each question has its own Q. Matrix D is of shape 300 x m+1 where m is the number of words in the document. Matrix Q is of shape 300 x n+1 where n is the number of wordsd in the question. 300 is the length of the word vectors, and the +1 comes from the sentinel vector. Let's assume we have these matrices. For now we'll use dummies of the right shape to get the process right. Also, we'll start by doing this for one document and one question. Later, we'll extend this to the whole dataset.

In [3]:
n = 20
m = 40
l = 200

Q = Variable(torch.Tensor(l, n+1).zero_(), requires_grad=False)
D = Variable(torch.Tensor(l, m+1).zero_(), requires_grad=False)
# We don't need to calculate gradients for this part of the process, right?"

In [4]:
# Calculate affinity matrix
L = D.transpose(0, 1).matmul(Q)

In [5]:
# Calculate attention weights A for Q and D
softmax = nn.Softmax(dim=1)
A_q = softmax(L)
A_d = softmax(L.transpose(0, 1))


In [6]:
# Compute attention context of the document wrt each word in the question
C_q = D.matmul(A_q)

In [7]:
# Compute attention context of the question wrt each word in the document and
# compute the attention context of the previous attention context wrt each
# word in the document. This is the coattention context C_d.
C_d = torch.cat((Q, C_q), dim=0).matmul(A_d)

In [8]:
class CoattentionEncoderBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(CoattentionEncoderBiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            bidirectional=True)
        
    def forward(self, x):
        out = self.lstm(x)
        return out

In [9]:
# Setup parameters

input_size = 600
hidden_size = 200
num_layers = 2
batch_size = 32
learning_rate = 0.0007
num_epochs = 2

In [11]:
# Setup model
model = CoattentionEncoderBiLSTM(input_size, hidden_size, num_layers)
model.cuda()
lossfun = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
for epoch in range(num_epochs):
    
    x = x.cuda()
    y = x
    output = model(x)
    optimizer.zero_grad()
    loss = lossfun(output[0], y)
    loss.backward()
    optimizer.step()

NameError: name 'x' is not defined